<a href="https://colab.research.google.com/github/gbrao018/eva4/blob/master/Assignment2_Ganji-step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1) #input -? OUtput? Local RF=(3*3), input:(28*28*1),out=(28*28*32), RF=3*3 , input channels=1, output channels=32
        self.batch1=nn.BatchNorm2d(4)
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1) #in=(28*28*32), out=(28*28,64),RF=5*5, input channels=32,output channels=64. Earlier 32 and current 64  kernels are working on same pixel only. RF not changing. 
        self.batch2=nn.BatchNorm2d(num_features=8)
        self.pool1 = nn.MaxPool2d(2, 2) # intput=(28*28,64), output=(14*14*64), Global RF (10*10). 
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) # input=(14*14*64), output=(14*14*128),RF=(12*12)
        self.batch3=nn.BatchNorm2d(num_features=16)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1) #input=(14*14*128) , output=(14*14*256),RF=(14*14)
        self.batch4=nn.BatchNorm2d(num_features=32)
        self.pool2 = nn.MaxPool2d(2, 2) # input=(14*14*256), output=(7*7*256),RF=(28*28)
        self.conv5 = nn.Conv2d(32, 32, 3) # input=(7*7*256), output=(5*5*512), no padding here,RF=(30*30)
        self.batch5=nn.BatchNorm2d(num_features=32)
        self.conv6 = nn.Conv2d(32, 32, 3) # input=(7*7*256), output=(5*5*512), no padding here,RF=(30*30)
        self.batch6=nn.BatchNorm2d(num_features=32)

        self.conv7 = nn.Conv2d(32, 10, 3) # input=(7*7*256), output=(5*5*512), no padding here,RF=(30*30)
        

    def forward(self, x):
        x = self.pool1(self.batch2(F.relu(self.conv2(self.batch1(F.relu(self.conv1(x)))))))
        x = self.pool2(self.batch4(F.relu(self.conv4(self.batch3(F.relu(self.conv3(x)))))))
        x = self.batch6(F.relu(self.conv6(self.batch5(F.relu(self.conv5(x))))))
        x=self.conv7(x)
        
        x = x.view(-1, 10)
        y=F.log_softmax(x)
        return F.log_softmax(x)

In [44]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
       BatchNorm2d-2            [-1, 4, 28, 28]               8
            Conv2d-3            [-1, 8, 28, 28]             296
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           1,168
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 32, 14, 14]           4,640
       BatchNorm2d-9           [-1, 32, 14, 14]              64
        MaxPool2d-10             [-1, 32, 7, 7]               0
           Conv2d-11             [-1, 32, 5, 5]           9,248
      BatchNorm2d-12             [-1, 32, 5, 5]              64
           Conv2d-13             [-1, 32, 3, 3]           9,248
      BatchNorm2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [11]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


# New Section

# New Section

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [45]:
#1. increased the epochs to 5, got 90% accuracy. Increasing further did not bring change
#2. no of params after
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0424, Accuracy: 9875/10000 (99%)




  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0351, Accuracy: 9885/10000 (99%)




  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0268, Accuracy: 9932/10000 (99%)




loss=0.00020687778305727988 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 36.72it/s]

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0280, Accuracy: 9931/10000 (99%)




loss=0.00015540917229373008 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 33.28it/s]


Test set: Average loss: 0.0277, Accuracy: 9930/10000 (99%)

